# a) Get number of voxels

In [ ]:
import os
import numpy as np # Just for refrences values

folder_path = "./3d_obj"

# Counts number of voxels
def manual_voxel_count(filepath):
    total = 0

    with open(filepath, "r") as f:
        for line in f:
            line = line.strip()

            # Count characters
            for char in line:
                if char == '1':
                    total += 1

    return total

# Python built-in library for reference
def library_voxel_count(filepath):
    with open(filepath, "r") as f:
        lines = [line.strip() for line in f if line.strip()]

    data = np.array([[int(c) for c in line] for line in lines])

    return np.sum(data)

# Main
for filename in os.listdir(folder_path):

    if filename.endswith(".txt"):

        filepath = os.path.join(folder_path, filename)

        print("\nProcessing:", filename)

        # manual count
        manual_count = manual_voxel_count(filepath)

        # library reference
        library_count = library_voxel_count(filepath)

        print("Manual voxel count:   ", manual_count)
        print("Library voxel count:  ", int(library_count))

        if manual_count == library_count:
            print("Counts are the same in the python library and the manual version")
        else:
            print("Counts don't match. There is an error.")


Processing: brain.txt
Manual voxel count:    4649382
Library voxel count:   4649382
Counts are the same in the python library and the manual version

Processing: bunny.txt
Manual voxel count:    3362339
Library voxel count:   3362339
Counts are the same in the python library and the manual version

Processing: cow.txt
Manual voxel count:    789300
Library voxel count:   789300
Counts are the same in the python library and the manual version

Processing: golfball.txt
Manual voxel count:    8700052
Library voxel count:   8700052
Counts are the same in the python library and the manual version

Processing: horse.txt
Manual voxel count:    717381
Library voxel count:   717381
Counts are the same in the python library and the manual version

Processing: igea.txt
Manual voxel count:    4766985
Library voxel count:   4766985
Counts are the same in the python library and the manual version

Processing: lion.txt
Manual voxel count:    501236
Library voxel count:   501236
Counts are the same in

# Get centroids

In [ ]:
import os

folder_path = "./3d_obj"
GRID_SIZE = 256

# Function to get center of mass
def manual_center_of_mass(filepath):

    sum_x = 0
    sum_y = 0
    sum_z = 0
    count = 0

    with open(filepath, "r") as f:

        layer = 0
        row = 0

        for line in f:
            line = line.strip()

            if not line:
                continue

            # Iterate through columns
            for col in range(len(line)):

                if line[col] == '1':

                    # Coordinate accumulation
                    sum_x += layer
                    sum_y += row
                    sum_z += col
                    count += 1

            row += 1

            # Every 256 rows there's a new layer
            if row == GRID_SIZE:
                row = 0
                layer += 1

    if count == 0:
        return (0,0,0,0)

    center_x = sum_x / count
    center_y = sum_y / count
    center_z = sum_z / count

    return (center_x, center_y, center_z, count)

# Numpy function to use for reference. Let's compare and see if the manual version was right.
def numpy_center_of_mass(filepath):

    with open(filepath, "r") as f:
        lines = [line.strip() for line in f if line.strip()]

    data = np.array([[int(c) for c in line] for line in lines])
    volume = data.reshape(GRID_SIZE, GRID_SIZE, GRID_SIZE)

    pts = np.argwhere(volume == 1)

    centroid = pts.mean(axis=0)

    return centroid

# Main
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        filepath = os.path.join(folder_path, filename)

        print("\nProcessing:", filename)

        # Manual results
        cx, cy, cz, total = manual_center_of_mass(filepath)

        # Numpy results for reference
        np_centroid = numpy_center_of_mass(filepath)

        #print("Total voxels:", total)

        print("\nManual centroid results:")
        print("X:", cx, "Y:", cy, "Z:", cz)

        #print("\nNumPy centroid results (for reference):")
        #print("X:", np_centroid[0],
              #"Y:", np_centroid[1],
              #"Z:", np_centroid[2])


Processing: brain.txt
Total voxels: 4649382

Manual centroid results:
X: 118.69456069645385 Y: 104.33795996973362 Z: 126.67544804879444

NumPy centroid results (for reference):
X: 118.69456069645385 Y: 104.33795996973362 Z: 126.67544804879444

Processing: bunny.txt
Total voxels: 3362339

Manual centroid results:
X: 120.76214177095171 Y: 164.5689545283804 Z: 119.14089120698418

NumPy centroid results (for reference):
X: 120.76214177095171 Y: 164.5689545283804 Z: 119.14089120698418

Processing: cow.txt
Total voxels: 789300

Manual centroid results:
X: 105.18711896617256 Y: 66.89249081464588 Z: 41.20030153300393

NumPy centroid results (for reference):
X: 105.18711896617256 Y: 66.89249081464588 Z: 41.20030153300393

Processing: golfball.txt
Total voxels: 8700052

Manual centroid results:
X: 127.40518091156237 Y: 127.41368350442043 Z: 127.49959483000791

NumPy centroid results (for reference):
X: 127.40518091156237 Y: 127.41368350442043 Z: 127.49959483000791

Processing: horse.txt
Total v

# b) Número de voxeles de la superficie

In [18]:
import os

SIZE = 256

# Check if voxel is outline
def is_surface(lines, line_idx, x):

    if lines[line_idx][x] != "1":
        return False

    z = line_idx // SIZE
    y = line_idx % SIZE

    neighbors = [
        (1,0,0), (-1,0,0),
        (0,1,0), (0,-1,0),
        (0,0,1), (0,0,-1)
    ]

    for dx, dy, dz in neighbors:

        nx = x + dx
        ny = y + dy
        nz = z + dz

        # If outside dimensions,, then outline voxel
        if nx < 0 or ny < 0 or nz < 0:
            return True
        if nx >= SIZE or ny >= SIZE or nz >= SIZE:
            return True

        n_line = nz*SIZE + ny

        if lines[n_line][nx] == "0":
            return True

    return False

# Count outline voxels in file
def count_surface_voxels(filepath):

    with open(filepath, "r") as f:
        lines = [line.strip() for line in f]

    count = 0

    for line_idx in range(len(lines)):
        row = lines[line_idx]

        for x in range(SIZE):
            if is_surface(lines, line_idx, x):
                count += 1

    return count

# Main
folder_path = "./3d_obj"

for filename in os.listdir(folder_path):

    if filename.endswith(".txt"):

        full_path = os.path.join(folder_path, filename)

        print("Currently at:", filename)

        surface_count = count_surface_voxels(full_path)

        print("Outline voxels:", surface_count)
        print("\n")


Currently at: brain.txt
Outline voxels: 206225


Currently at: bunny.txt
Outline voxels: 135743


Currently at: cow.txt
Outline voxels: 54938


Currently at: golfball.txt
Outline voxels: 171938


Currently at: horse.txt
Outline voxels: 58294


Currently at: igea.txt
Outline voxels: 131864


Currently at: lion.txt
Outline voxels: 52094


Currently at: lucy.txt
Outline voxels: 52246


Currently at: pear.txt
Outline voxels: 77584


Currently at: torus.txt
Outline voxels: 87484


